In [184]:
import cv2
import numpy as np

def nasyc(img):
    [[cell for cell in row] for row in img][2]=255
    return img

def main():
    kernel = np.ones((2,2),np.uint8)
    tablice = []
    for i in range(1,17):
        tablice.append("Samochody_zrodlo/sam" + str(i) + ".png")
    for tablica in tablice:
        print(tablica)
        zdjecie=cv2.imread(tablica)
        
        zdjecie=cv2.cvtColor(zdjecie,cv2.COLOR_BGR2HSV)
        #cv2.imshow("begin", nasyc(zdjecie))
        
        low=np.array([0,0,25])
        high=np.array([360,50,250])   
        low2=np.array([100,60,20])
        high2=np.array([130,255,250])
        
        maska=cv2.inRange(zdjecie,low,high)
        maska2=cv2.inRange(zdjecie,low2,high2)
        
        res=cv2.bitwise_and(zdjecie,zdjecie,mask=maska)
        res=cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
        
        print(res[0][0])
        
        res2=cv2.bitwise_and(zdjecie,zdjecie,mask=maska2)
        res2=cv2.cvtColor(res2,cv2.COLOR_HSV2BGR)
        
        cv2.imwrite("Samochody/B" + tablica, res )
        cv2.imwrite("Samochody/N" + tablica, res2 )
        #res,thresh=cv2.threshold(res,128,255,cv2.THRESH_BINARY)
        #thresh=res
        #thresh=cv2.morphologyEx(res,cv2.MORPH_CLOSE,kernel)
        #cv2.imshow("thresh000",thresh)
        #thresh=cv2.Canny(thresh,len(thresh),len(thresh[0]))

    
        #cv2.imshow("zdjecie",zdjecie)
        #cv2.imshow("maska",maskafinal)
        #cv2.imshow("maska2",maska2)
        #cv2.imshow("res",res)
        #cv2.imshow("thresh",thresh)
        
        #cv2.waitKey(0)
    
    
if __name__=="__main__":
    main()

Samochody_zrodlo/sam1.png
[139 145 152]
Samochody_zrodlo/sam2.png
[180 197 193]
Samochody_zrodlo/sam3.png
[76 83 78]
Samochody_zrodlo/sam4.png
[74 71 67]
Samochody_zrodlo/sam5.png
[139 133 128]
Samochody_zrodlo/sam6.png
[0 0 0]
Samochody_zrodlo/sam7.png
[83 85 85]
Samochody_zrodlo/sam8.png
[0 0 0]
Samochody_zrodlo/sam9.png
[58 62 67]
Samochody_zrodlo/sam10.png


error: D:\Build\OpenCV\opencv-3.3.1\modules\imgproc\src\color.cpp:11115: error: (-215) (scn == 3 || scn == 4) && (depth == 0 || depth == 5) in function cv::cvtColor
